## Merge features and labels using SQL
This notebook is for merging my features from the audio files to the labels. Both are imported as csvs to pandas. I could merge them directly in pandas, but am using SQL (Postgres) to demonstrate the skillset. 

In [49]:
import pickle
import pandas as pd
import numpy as np
import psycopg2 as pg
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas.io.sql as pd_sql
import matplotlib.pyplot as plt
%matplotlib inline

## Read in DataFrame as df

In [7]:
df = pd.read_csv('complete_data_unlabeled.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 31 columns):
Unnamed: 0            30000 non-null int64
centroid              30000 non-null object
chroma                30000 non-null object
chroma_cens           30000 non-null object
chroma_cqt            30000 non-null object
contrast              30000 non-null object
file_number           30000 non-null int64
flatness              30000 non-null object
index                 30000 non-null object
mean_centroid         30000 non-null float64
mean_chroma           30000 non-null float64
mean_chroma_cens      30000 non-null float64
mean_chroma_cqt       30000 non-null float64
mean_contrast         30000 non-null float64
mean_flatness         30000 non-null float64
mean_mel              30000 non-null float64
mean_mfccs            30000 non-null float64
mean_rmse             30000 non-null float64
mean_rolloff          30000 non-null float64
mean_spec_bw          30000 non-null float64
me

### Read labels csv

In [9]:
labels = pd.read_csv('train.csv')
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
clip_name    30000 non-null object
label        30000 non-null int64
dtypes: int64(1), object(1)
memory usage: 468.8+ KB


In [10]:
labels['file_number'] = labels.clip_name.apply(lambda x: str(x.split('.')[0]).split('n')[1]) 

In [11]:
labels.tail()

,clip_name,label,file_number
29995,train29996.aiff,0,29996
29996,train29997.aiff,0,29997
29997,train29998.aiff,0,29998
29998,train29999.aiff,0,29999
29999,train30000.aiff,0,30000


In [30]:
labels['file_number'] = labels['file_number'].astype(int)

In [31]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
clip_name      30000 non-null object
label          30000 non-null int64
file_number    30000 non-null int64
dtypes: int64(2), object(1)
memory usage: 703.2+ KB


## Postgres

In [1]:
params = {
    'host': '127.0.0.1', # previously 18.220.115.81
    #'user': 'ubuntu',
    'port': 5432,
    'user': 'kmussar',
    'password':1234
}

# Connect
connection = connect(**params) 
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [2]:
# Create the whales database
connection.cursor().execute('CREATE DATABASE whales;')
connection.close()

In [13]:
# Create the features (data) table
from sqlalchemy import create_engine
connection_string = f'postgres://{params["user"]}:{params["password"]}@{params["host"]}:{params["port"]}/whales'
engine = create_engine(connection_string)
df.to_sql('features', engine, index=False)

In [32]:
# create a labels table
labels.to_sql('labels2', engine, index=False)

In [17]:
# Set up cursor
connection = connect(**params, dbname='whales')
cursor = connection.cursor()
# test that database and tables are working
cursor.execute("SELECT * FROM features WHERE features.file_number < 100;")
cursor.fetchall()

[(0,
  '[[504.48257148 486.40693278 470.00318888 458.39376127 463.90379466\n  477.39413771 471.14403268 471.00180014 460.68352946 453.9339167\n  472.69338205 489.12105578 489.61289055 488.05017456 484.86728061\n  479.53027088 489.12357299 487.65071429 461.64060034 460.1356\n  433.91646698 429.06201334 424.18437903 441.81792366 440.54207934\n  449.78643017 478.41340771 470.58650832 456.33182708 428.39037465\n  427.35647774 447.83947686 443.543329   445.80229821 479.36827219\n  418.27833378 370.90340022 392.98444249 439.0988273  451.57873422\n  462.15542552 443.66547733 422.60334107 427.49937999 390.81601901\n  363.93926919 385.9929467  437.78295228 453.37385921 449.99199613\n  422.09881158 418.24084294 402.52489621 397.39166578 383.29910392\n  383.10982157 417.95844503 427.78726583 438.653755   436.30790811\n  409.32322255 402.25753494 433.26452727 427.89675082 429.53106227\n  433.35584636 423.51103225 436.69896297 419.21593564 436.11478736\n  435.24388274 450.43534722 443.91212624 437.

## Merge subset of dataframe as a test

In [54]:
# join tables on file_number
connection = connect(**params, dbname='whales')
cursor = connection.cursor()

query = """CREATE VIEW labeled_data AS (
SELECT mean_rmse, label FROM features 
JOIN labels2 ON features.file_number = labels2.file_number);"""
cursor.execute(query)

In [55]:
query = """SELECT * FROM labeled_data LIMIT 5;"""
cursor.execute(query)
cursor.fetchall()

[(0.328017890453339, 0),
 (0.11957024037838, 0),
 (0.153038755059242, 0),
 (0.16910295188427, 0),
 (0.222452014684677, 0)]

In [56]:
query = """SELECT * FROM labeled_data LIMIT 5;"""
test_df = pd_sql.read_sql(query, connection)
test_df

,mean_rmse,label
0,0.328018,0
1,0.119570,0
2,0.153039,0
3,0.169103,0
4,0.222452,0


## Merge whole dataframe

In [57]:
# join tables on file_number
connection = connect(**params, dbname='whales')
cursor = connection.cursor()

query = """CREATE VIEW labeled_data AS (
SELECT features.*, labels2.label FROM features 
JOIN labels2 ON features.file_number = labels2.file_number);"""
cursor.execute(query)

In [60]:
query = """SELECT * FROM labeled_data;"""
labeled_df = pd_sql.read_sql(query, connection)
labeled_df.head()

,Unnamed: 0,centroid,chroma,chroma_cens,chroma_cqt,contrast,file_number,flatness,index,mean_centroid,...,mean_zero_cr,mel,mfccs,rmse,rolloff,spec_bw,spec_contrast,tonnetz,zero_cr,label
0,0,[[504.48257148 486.40693278 470.00318888 458.3...,[0.54581436 0.60204302 0.75422708 0.83723864 0...,[[0.23143955 0.23162181 0.23182812 ... 0.27899...,[[0.55334989 0.73690282 0.70845463 ... 0.70770...,[19.62451933 11.26503332 12.72140427 56.165287...,1,[[1.32692594e-06 3.85900563e-07 8.42322656e-09...,train1.wav,443.787989,...,0.042138,[4.66567711e-02 1.67171275e-01 3.63310730e-01 ...,[-6.44730806e+02 2.56817895e+02 1.58503050e+...,[[0.26807293 0.2719934 0.30389765 0.31095374 ...,[[753.66210938 753.66210938 742.89550781 721.3...,[[330.53144218 281.77145121 245.76683196 239.8...,[[22.44497389 10.83755733 14.62975989 20.37279...,[ 0.00198343 -0.00508595 0.0660235 0.047146...,[[0.02392578 0.03466797 0.04541016 0.046875 ...,0
1,1,[[577.97243044 524.01977987 427.78777341 420.3...,[0.6937604 0.6619282 0.67327483 0.83409719 0...,[[0.21704897 0.21644774 0.21590343 ... 0.29807...,[[0.91323336 0.72939455 0.66281015 ... 1. ...,[16.21541347 10.36397665 13.25033723 54.129334...,2,[[6.5010463e-05 2.9992112e-05 4.2579142e-08 5....,train2.wav,463.286988,...,0.042093,[1.24066644e-04 7.28846028e-04 2.67229303e-03 ...,[-7.36124813e+02 2.44485183e+02 1.49323090e+...,[[0.17244595 0.13339429 0.11844186 0.10560928 ...,[[753.66210938 732.12890625 667.52929688 645.9...,[[663.20235137 548.26586027 208.22355781 203.6...,[[11.80591084 16.68927063 13.92415427 16.83627...,[ 0.00818045 -0.00117309 0.02137063 0.005662...,[[0.02197266 0.03417969 0.04345703 0.04101562 ...,0
2,2,[[559.55729635 528.68537997 493.25946834 479.8...,[0.67377754 0.67545543 0.76603449 0.75670351 0...,[[0.36211961 0.3571492 0.35205534 ... 0.28488...,[[0.57887618 0.8278652 0.80876065 ... 0.44839...,[19.49029831 10.76560549 12.28624703 54.706529...,3,[[5.3348005e-05 1.6811937e-05 3.5331706e-08 2....,train3.wav,468.395865,...,0.042868,[1.69410231e-04 8.37843759e-04 1.69996183e-03 ...,[-7.18175685e+02 2.49495845e+02 1.51575915e+...,[[0.14040385 0.1359778 0.13274789 0.14748028 ...,[[732.12890625 732.12890625 742.89550781 710.5...,[[663.76955723 480.21842727 204.16179659 191.7...,[[12.65364156 8.33372768 14.72258296 13.54955...,[ 0.005731 -0.0008887 0.01147021 0.010365...,[[0.02392578 0.03613281 0.04541016 0.04638672 ...,0
3,3,[[476.40339026 466.14735071 486.65554654 493.1...,[0.71536 0.68535892 0.68436785 0.73879001 0...,[[0.38081637 0.37959923 0.3784084 ... 0.28371...,[[0.71287569 0.91182481 0.81314382 ... 0.84112...,[21.99398376 11.21640425 13.21343611 56.224961...,4,[[3.93400853e-07 1.46186906e-07 3.61700003e-08...,train4.wav,456.602276,...,0.040134,[9.53286843e-05 3.84619199e-04 1.28008202e-03 ...,[-7.10932459e+02 2.44696894e+02 1.51166062e+...,[[0.13967143 0.13067499 0.13031767 0.16876325 ...,[[742.89550781 721.36230469 732.12890625 678.2...,[[269.38864711 239.29599281 209.14325354 192.6...,[[25.21498022 11.79400243 19.01732861 24.72397...,[ 0.00315338 0.00467927 0.02231854 0.013073...,[[0.02294922 0.03515625 0.04736328 0.04589844 ...,0
4,4,[[549.73110503 506.98113396 473.99013424 473.9...,[0.62032704 0.61695566 0.67535189 0.69344183 0...,[[0.29312185 0.29195579 0.2914121 ... 0.29096...,[[0.43504997 0.84417333 0.96267805 ... 0.20194...,[21.30596447 10.20325473 12.75112812 55.260719...,5,[[3.01305136e-05 8.84687415e-06 1.99340775e-08...,train5.wav,471.288952,...,0.043244,[8.71598585e-05 2.49764398e-04 9.07598442e-04 ...,[-6.60409241e+02 2.30303621e+02 1.40471045e+...,[[0.19937678 0.19993429 0.18210486 0.20334208 ...,[[699.82910156 721.36230469 732.12890625 721.3...,[[575.75803571 421.59927487 214.78639831 194.3...,[[13.6326702 12.90958534 22.18394664 20.01080...,[ 0.00511436 0.01517265 -0.00726036 0.016294...,[[0.02099609 0.03076172 0.04150391 0.04052734 ...,0


In [61]:
labeled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 32 columns):
Unnamed: 0            30000 non-null int64
centroid              30000 non-null object
chroma                30000 non-null object
chroma_cens           30000 non-null object
chroma_cqt            30000 non-null object
contrast              30000 non-null object
file_number           30000 non-null int64
flatness              30000 non-null object
index                 30000 non-null object
mean_centroid         30000 non-null float64
mean_chroma           30000 non-null float64
mean_chroma_cens      30000 non-null float64
mean_chroma_cqt       30000 non-null float64
mean_contrast         30000 non-null float64
mean_flatness         30000 non-null float64
mean_mel              30000 non-null float64
mean_mfccs            30000 non-null float64
mean_rmse             30000 non-null float64
mean_rolloff          30000 non-null float64
mean_spec_bw          30000 non-null float64
me

In [62]:
labeled_df.drop(columns=['Unnamed: 0'],axis=1, inplace=True)

In [63]:
labeled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 31 columns):
centroid              30000 non-null object
chroma                30000 non-null object
chroma_cens           30000 non-null object
chroma_cqt            30000 non-null object
contrast              30000 non-null object
file_number           30000 non-null int64
flatness              30000 non-null object
index                 30000 non-null object
mean_centroid         30000 non-null float64
mean_chroma           30000 non-null float64
mean_chroma_cens      30000 non-null float64
mean_chroma_cqt       30000 non-null float64
mean_contrast         30000 non-null float64
mean_flatness         30000 non-null float64
mean_mel              30000 non-null float64
mean_mfccs            30000 non-null float64
mean_rmse             30000 non-null float64
mean_rolloff          30000 non-null float64
mean_spec_bw          30000 non-null float64
mean_spec_contrast    30000 non-null float64


In [64]:
with open('all_labeled_data.pickle', 'wb') as to_write:
    pickle.dump(labeled_df, to_write) 